<a href="https://colab.research.google.com/github/sadiashoaib415/Quantum_Computer/blob/main/Quantum_Chemistry_Qiskit_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install the Quantum and Chemistry stacks
!pip install qiskit qiskit-aer qiskit-algorithms qiskit-nature[pyscf] -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 47.0 MB/s eta 0:00:00


In [2]:
import numpy as np
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import StatevectorEstimator

# 1. Define the Molecule (H2 at its equilibrium distance)
driver = PySCFDriver(atom="H 0 0 0; H 0 0 0.735", basis="sto3g")
problem = driver.run()

# 2. Setup the "Translator" (Fermions to Qubits)
mapper = JordanWignerMapper()

# 3. Extract the Hamiltonian (The Energy Equation)
hamiltonian = problem.hamiltonian.second_q_op()
qubit_op = mapper.map(hamiltonian)

print("Molecule defined and mapped to qubits.")

Molecule defined and mapped to qubits.


In [3]:
# 1. Build the Ansatz (The Quantum 'Guess' Circuit)
ansatz = UCCSD(
    problem.num_spatial_orbitals,
    problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
    ),
)

# 2. Setup the Solver (VQE) using the modern Estimator
estimator = StatevectorEstimator()
optimizer = SLSQP(maxiter=100)
vqe = VQE(estimator, ansatz, optimizer)

# 3. Solve for the Ground State
result = vqe.compute_minimum_eigenvalue(qubit_op)

# 4. Interpret the result (Electronic Energy + Nuclear Push)
total_energy = result.eigenvalue + problem.nuclear_repulsion_energy

print(f"--- Quantum Chemistry Result ---")
print(f"Ground State Energy: {total_energy:.4f} Hartree")

/usr/local/lib/python3.12/dist-packages/scipy/sparse/linalg/_dsolve/linsolve.py:606: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
/usr/local/lib/python3.12/dist-packages/scipy/sparse/linalg/_matfuncs.py:707: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  return spsolve(Q, P)


--- Quantum Chemistry Result ---
Ground State Energy: -1.1373 Hartree
